# TITANIC - KAGGLE

## DATA PREPARATION

#### LIBRARY IMPORTS 

We will the `pandas` library to inspect data.

In [1]:
import pandas as pd

If data isn't already downloaded, uncomment the following and download via the KAGGLE API.

In [2]:
# !kaggle competitions download -c "titanic"

Move the `titanic.zip` file to the folder `datasets/`. Unzip that file and move unzipped files to `datasets/`.

In [3]:
# ! mv titanic.zip datasets/
# ! unzip datasets/titanic.zip
# ! mv gender_submission.csv datasets/
# ! mv test.csv datasets/
# ! mv train.csv datasets/

Read the training and test data files into a pandas dataframe.

In [4]:
trainData = pd.read_csv('datasets/train.csv')
testData = pd.read_csv('datasets/test.csv')

### PRELIMINARY DATA ANALYSIS

Before constructing and implemeting a model, it is necessary to inspect the data sources to sift for relevant and sift out irrelevant data. Note, both the training and test data must have the same data *fields* when implementing a model. We can however remove specific entries from the training data without altering the test data. If we remove an entire column from the training data however, this same column must be removed from the test data.

#### TRAINING DATA

The following reveals the data fields with `null` or `NaN` entries in the training data.

In [5]:
for col in trainData.columns:
    print(len(trainData[trainData[col].isnull()]))

0
0
0
0
0
177
0
0
0
0
687
2


And more specifically:

In [6]:
trainlength = len(trainData)
for col in trainData.columns:
    L = len(trainData[trainData[col].isnull()])
    if L > 0:
        percent = round(100*L/trainlength, 2)
        print(f"{col} column has {L} many null or NaN values. This represents {percent} percent of the training data.")
    

Age column has 177 many null or NaN values. This represents 19.87 percent of the training data.
Cabin column has 687 many null or NaN values. This represents 77.1 percent of the training data.
Embarked column has 2 many null or NaN values. This represents 0.22 percent of the training data.


#### TEST DATA

Similarly for test data we have:

In [7]:
testLength = len(testData)
for col in testData.columns:
    L = len(testData[testData[col].isnull()])
    if L > 0:
        percent = round(100*L/testLength, 2)
        print(f"{col} column has {L} many null or NaN values. This represents {percent} percent of the testing data.")


Age column has 86 many null or NaN values. This represents 20.57 percent of the testing data.
Fare column has 1 many null or NaN values. This represents 0.24 percent of the testing data.
Cabin column has 327 many null or NaN values. This represents 78.23 percent of the testing data.


### PRELIMINARY CONCLUSIONS 

There is missing data in both training and test datasets. To handle this we can look to either:

- fill in the data; 
- exclude it;
- train different models and apply them to the cases where data is missing or is not.

We look at each field with missing data seperately.

#### AGE

The `Age` datafield has around `20%` of unfilled values over both the training and test data. This is typically not enough to warrent excluding `Age` entirely. A better option might be to used a mixed model approach: to train a model including `Age` and one excluding `Age`. Then apply one of the two models on the testing data if `Age` is known or not.  

In this notebook we will simply exclude `Age`.  

#### EMBARKED

The test data does not have any missing values for `Embarked`. As only around `0.2%` of these values are missing in the training dataset, entries with missing `Embarked` value can be excluded in training.

#### CABIN

Over both the training and test data, see that almost `80%` of`Cabin` data is not recorded. It would be difficult to fill in this much of data given only `20%` and so this field is likely to have little influence on the final prediction. We will exclude the `Cabin` datafield.

#### FARE

There are no empty values for `Fare` in the training data and only `0.24%` of these values are missing in the test data. If we were to exclude this field from the test data, we would have to exclude it from the training data. We are better off simply filling it in the test data. We can do this by replacing all `null` values with the `Fare` datafield average. 

**Remark.** *For a slightly more accurate fill, we could use that `Fare` is probably dependent on passenger class. Since the passenger class, `Pclass` is known, we can fill in the `Fare` datafield by the average of this datafield for the given `Pclass`. E.g., if `Pclass` for the missing fare is `1`, we can look at the average fare for first class passengers and use this value for the missing fare.*


### DATA PREPARATION

#### REMOVAL

Based on the preliminary conclusion reached above, we will remove unecessary columns and ensure that any categorical data is of numerical type. The columns with their data types are: 

In [8]:
for col in list(trainData.columns):
    print(f"{col}: has data type {trainData[col].dtypes}")

PassengerId: has data type int64
Survived: has data type int64
Pclass: has data type int64
Name: has data type object
Sex: has data type object
Age: has data type float64
SibSp: has data type int64
Parch: has data type int64
Ticket: has data type object
Fare: has data type float64
Cabin: has data type object
Embarked: has data type object


Columns which we expect would *not* be useful in predicting survival are `PassengerId`, `Name` and `Ticket`. 

**Remark.** *One could imagine `Name` to be useful however if it contained prefixes such as `Dr.`, `Mr.`, `Mrs.` as people with such titles might influence their chance of survival. In preparing the data in this notebook however we do not assume this and consider `Name` irrelevant.*

We can remove these columns from both datasets. We will retain `PassengerId` in the test data however since, in the final submission of predictions, we are required to submit passenger survivor predictions along with their ID. 

In [9]:
trainData = trainData.drop(["PassengerId", "Name", "Ticket", "Age", "Cabin"], axis=1)
testData = testData.drop(["Name", "Ticket", "Age", "Cabin"], axis=1)

In [10]:
print(trainData.columns)
print(testData.columns)

Index(['Survived', 'Pclass', 'Sex', 'SibSp', 'Parch', 'Fare', 'Embarked'], dtype='object')
Index(['PassengerId', 'Pclass', 'Sex', 'SibSp', 'Parch', 'Fare', 'Embarked'], dtype='object')


Reindexing:

In [11]:
trainData = trainData.reset_index(drop=True)
testData = testData.reset_index(drop=True)

### DATA NUMERICIZATION

The training and test data contain both numerical and categorical data. The categorical data can be mapped to numbers through the method of `one-hot encoding`, e.g., `Sex` can be encoded by `0` or `1` according to categorical values `male` and `female` and similarly for `Embarked`.

In the approach of `one-hot encoding`, each category appears as a separate column in the dataset. The value of `1` or `0` is recorded according to whether this property is true or false in the entry respectively.

Doing this for the training and test data below gives a new, one-hot encoded numericized dataset.

In [12]:
trainNumerCols = [
            'Survived',
            'Pclass',
            "male",
            "female",
            'SibSp',
            'Parch',
            'Fare',
            "embarkedC",
            "embarkedS",
            "embarkedQ"
        ]

list(trainData.columns)
trainNumer = pd.DataFrame(columns = trainNumerCols)

for i, row in trainData.iterrows():
    
    male = 0
    if row["Sex"] == 'male':
        male = 1
    
    female = 0
    if row['Sex'] == 'female':
        female = 1
        
    embC = 0
    if row['Embarked'] == 'C':
        embC = 1
        
    embS = 0
    if row['Embarked'] == 'S':
        embS = 1
    
    embQ = 0
    if row['Embarked'] == 'Q':
        embQ = 1

    trainNumer = trainNumer.append(pd.Series(
        [
            row['Survived'],
            row['Pclass'],
            male,
            female,
            row['SibSp'],
            row['Parch'],
            row['Fare'],
            embC,
            embS,
            embQ
        ], index = trainNumerCols
    ), ignore_index=True)


In [13]:
testNumerCols = [
    'PassengerId',
    'Pclass',
    'male',
    'female',
    'SibSp',
    'Parch',
    'Fare',
    'embarkedC',
    'embarkedS',
    'embarkedQ'
]
testNumer = pd.DataFrame(columns=testNumerCols)

for i, row in testData.iterrows():

    male = 0
    if row["Sex"] == 'male':
        male = 1
    
    female = 0
    if row['Sex'] == 'female':
        female = 1
        
    embC = 0
    if row['Embarked'] == 'C':
        embC = 1
        
    embS = 0
    if row['Embarked'] == 'S':
        embS = 1
    
    embQ = 0
    if row['Embarked'] == 'Q':
        embQ = 1
        
    if pd.isnull(testData["Fare"][i]):
        fare = testData["Fare"].mean()
    else:
        fare = row["Fare"]
    
    testNumer = testNumer.append(pd.Series(
        [
            row['PassengerId'],
            row['Pclass'],
            male,
            female,
            row['SibSp'],
            row['Parch'],
            fare,
            embC,
            embS,
            embQ
        ], index = testNumerCols
    ), ignore_index=True)


### OUTPUT


Now that the datasets have been analysed and numericized, we can output them to `.csv` files, ready to be parsed by a machine learning model. For supervised machine learning models we need to parse *training data*, *training data labels* and the *test data*. Validation, if specified, is implemented on training data.

#### TRAINING DATA OUTPUTS

Note, labels for training data is the column `Survived`. Accordingly, this is removed in forming the training data set.


In [14]:
trainingData = trainNumer.drop(['Survived'], axis=1)

Labels for the training data is the `Survived` column which we grab and output as a pandas dataframe.

In [15]:
labels = trainNumer["Survived"].to_frame()

There is nothing further to process concerning the test data.

#### TO CSV

In [18]:
trainingData.to_csv('datasets/trainingData.csv', index=False)
labels.to_csv('datasets/trainingLabels.csv', index=False)
testNumer.to_csv('datasets/testData.csv', index=False)